In [1]:
import numpy as np
import astropy.coordinates as astroCoords
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS

In [2]:
angle = 0.

In [3]:
vel_array = np.array([[6.*np.cos(angle), 6.*np.sin(angle)]])

In [4]:
vel_array

array([[ 6.,  0.]])

In [26]:
test_fits = fits.open('../../HITS/trimmed_chip_03/Blind15A_01/search_nights/trimmed_calexp-0410840_03.fits')
#time_array = np.genfromtxt('../../HITS/test/chip_05/field_times.dat')

In [27]:
#test_fits = fits.open('../../HITS/test/chip_05/calexp-0412680_05.fits')
#time_array = np.genfromtxt('../../HITS/test/chip_05/field_times.dat')

In [28]:
test_wcs = WCS(test_fits[1].header)

In [29]:
time_array = [0.0, 1.0, 2.0]

In [30]:
wcs = [test_wcs]
pixel_coords = [[],[]]
pixel_start = [[2000, 4000]]

vel_par_arr = vel_array[:, 0]
vel_perp_arr = vel_array[:, 1]

if type(vel_par_arr) is not np.ndarray:
    vel_par_arr = [vel_par_arr]
if type(vel_perp_arr) is not np.ndarray:
    vel_perp_arr = [vel_perp_arr]
for start_loc, vel_par, vel_perp in zip(pixel_start, vel_par_arr, vel_perp_arr):
    
    print start_loc
    start_coord = astroCoords.SkyCoord.from_pixel(start_loc[0],
                                                  start_loc[1],
                                                  wcs[0])
    eclip_coord = start_coord.geocentrictrueecliptic
    eclip_l = []
    eclip_b = []
    for time_step in time_array:
        eclip_l.append(eclip_coord.lon + vel_par*time_step*u.arcsec)
        eclip_b.append(eclip_coord.lat + vel_perp*time_step*u.arcsec)
    eclip_vector = astroCoords.SkyCoord(eclip_l, eclip_b,
                                        frame='geocentrictrueecliptic')
    pixel_coords_set = astroCoords.SkyCoord.to_pixel(eclip_vector, wcs[0])
    pixel_coords[0].append(pixel_coords_set[0])
    pixel_coords[1].append(pixel_coords_set[1])
pixel_coords = np.array(pixel_coords)

[2000, 4000]


In [31]:
pixel_coords

array([[[ 2000.        ,  2006.76629527,  2013.53267996]],

       [[ 4000.00000025,  4020.25041785,  4040.50084292]]])

In [32]:
x_dist = pixel_coords[0, 0, -1] - pixel_coords[0,0,0]
y_dist = pixel_coords[1,0,-1] - pixel_coords[1,0,0]

In [33]:
x_dist, y_dist

(13.53267996141085, 40.500842676173761)

In [35]:
np.degrees(np.arctan(y_dist/x_dist))

71.523810163542649